Setup

In [ ]:
pip install pandas openpyxl pillow numpy

In [ ]:
import pandas as pd
import openpyxl
from PIL import Image
from PIL import Image, ImageDraw, ImageChops, ImageFont
import os

In [ ]:
sheet = "2025"

df = pd.read_excel("data/Proto-2_Frame_Auto.xlsx", sheet_name=sheet)
df


### Filter Values Birthdays, Anniversaries, etc.

In [ ]:
# change value of this variable
month = "May"
bday = df[(df["birthday"] == month)]
bday_fields = bday[["short_name", "k-id"]]

anniv = df[(df["anniversary"] == month)]
anniv_fields = anniv[["short_name", "k-id"]]


In [ ]:
bday_test = bday_fields.to_dict(orient='split', index=False)
bday_data = bday_test['data']

anniv_test = anniv_fields.to_dict(orient='split', index=False)
anniv_data = anniv_test['data']

### Creating groups (to modify)

In [ ]:
bday_groups = [] 
while bday_data:
    n = len(bday_data)
    for size in range(8, 0, -1):
        if n % size == 0 or n >= size:
            bday_groups.append(bday_data[:size])
            bday_data = bday_data[size:]
            break


print("BIRTHDAY")
print("="*70)
for i, group in enumerate(bday_groups, 1):
    print(f"Group {i} ({len(group)} members):", group)




anniv_groups = [] 
while anniv_data:
    n = len(anniv_data)
    for size in range(8, 0, -1):
        if n % size == 0 or n >= size:
            anniv_groups.append(anniv_data[:size])
            anniv_data = anniv_data[size:]
            break

print()
print("Anniversary")
print("="*70)
for i, group in enumerate(anniv_groups, 1):
    print(f"Group {i} ({len(group)} members):", group)



### Image Coordinates

In [ ]:
single_y = 418
top_y = 328
bottom_y = 732

coord_arr_8 = [(51, top_y),(500, top_y),(948, top_y),(1396, top_y),
               (266, bottom_y),(714, bottom_y),(1163, bottom_y),(1611, bottom_y)]
coord_arr_7 = [(91, top_y),(589, top_y),(1087, top_y),(1585, top_y),
               (308, bottom_y),(838, bottom_y),(1367, bottom_y)]
coord_arr_6 = [(203, top_y),(733, top_y),(1262, top_y),
               (454, bottom_y),(983, bottom_y),(1513, bottom_y)]
coord_arr_5 = [(203, top_y), (838, top_y), (1472, top_y), 
               (518, bottom_y), (1158, bottom_y)]
coord_arr_4 = [(172, single_y),(616, single_y),(1060, single_y),(1504, single_y)]
coord_arr_3 = [(268, single_y),(838, single_y),(1408, single_y)]
coord_arr_2 = [(565, single_y),(1111, single_y)]
coord_arr_1 = [(838, single_y)]

## Notes

TODO: 
- calculate image and text offset to make the anchor point be the middle
- read the name and image path from the excel file 
- use the appropriate canvas when of number of people 

TODO: 
- Make this into a video, save it and get the duration 
- Set this duration in the software upload and observe how it will play

TODO:
- create folder for image and file in the downloads folder for easy access 
- detect if the file is an image or excel file 
- move them in their respective folders inside the project 

# Main Process

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os

# Values to change
outer_fill_color = "#ed6e38"
inner_fill_color = "#ffcf00"
font = ImageFont.truetype("DejaVuSansMono.ttf", size=25) # note: windows font starts with small letter
department = "IT"
category = "bday" # or bday
data_group = bday_groups # or bday_groups

# Coordinate mapping
coord_map = {
    8: coord_arr_8,
    7: coord_arr_7,
    6: coord_arr_6,
    5: coord_arr_5,
    4: coord_arr_4,
    3: coord_arr_3,
    2: coord_arr_2,
    1: coord_arr_1,
}

IMAGE_SIZE = 245

# Make an image circular with transparency
def make_circular(image_path):
    img = Image.open(image_path).convert("RGBA")
    width, height = img.size

    mask = Image.new("L", (width, height), 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0, width, height), fill=255)
    img.putalpha(mask)
    return img

def create_border(diamter, x0, y0):
    x1, y1 = x0 + diamter, y0 + diamter
    bounding_box = x0, y0, x1, y1 
    return bounding_box

def calculate_border(IMAGE_SIZE, image_x, image_y, size_difference, position_offset):
    diameter = IMAGE_SIZE + size_difference
    x_coord_diff = y_coord_diff = position_offset
    x_offset = image_x - x_coord_diff
    y_offset = image_y - y_coord_diff
    return diameter, x_offset, y_offset


# Create the final composite image
def create_image(names, coords, usernames, department, category, counter, background_path):
    canvas = Image.open(background_path).convert("RGBA")
    draw = ImageDraw.Draw(canvas)

    for (image_x, image_y), username, name in zip(coords, usernames, names):
        image_path = f"images/{username}.jpg"
        try:
            circular_image = make_circular(image_path)
        except FileNotFoundError:
            circular_image = make_circular("images/person-holder.jpg")

        # Setting values for inner and outer border size and coordinate
        INNER_BORDER_SIZE_OFFSET = 29
        INNER_BORDER_POSITION_OFFSET = 15
        OUTER_BORDER_SIZE_OFFSET = 63
        OUTER_BORDER_POSITION_OFFSET = 32
        
        inner_diameter, inner_x_offset, inner_y_offest = calculate_border(IMAGE_SIZE, image_x, image_y, 
                                                                          INNER_BORDER_SIZE_OFFSET, INNER_BORDER_POSITION_OFFSET)
        outer_diameter, outer_x_offset, outer_y_offest = calculate_border(IMAGE_SIZE, image_x, image_y,
                                                                          OUTER_BORDER_SIZE_OFFSET, OUTER_BORDER_POSITION_OFFSET)

        #  Applying the values and drawing it to the canvas
        outer_bounding_box = create_border(outer_diameter, outer_x_offset, outer_y_offest)
        draw.ellipse(outer_bounding_box, fill=outer_fill_color)

        inner_bounding_box = create_border(inner_diameter, inner_x_offset, inner_y_offest)
        draw.ellipse(inner_bounding_box, fill=inner_fill_color)

        resized_image = circular_image.resize((245, 245), Image.LANCZOS)
        canvas.paste(resized_image, (image_x, image_y), resized_image)

        # Configuring text values 
        TEXT_X_OFFSET = 120
        TEXT_Y_OFFSET = 300
        text_position = (image_x + TEXT_X_OFFSET, image_y + TEXT_Y_OFFSET)
        draw.text(text_position, name, fill=(0, 0, 0), font=font, anchor="mm")

    # Saving File
    output_name = f"[{department}]-{category}-{counter}.jpeg"
    os.makedirs("output", exist_ok=True)
    canvas.convert("RGB").save(os.path.join(f"output", output_name), "JPEG")


# Runner loop
counter = 0
for group in data_group:
    group_size = len(group)    
    coords = coord_map.get(group_size)
    background_path = f"backgrounds/{category}.png"
    if coords and background_path:
        usernames = [person[1].replace(".", "-") for person in group]
        names = [person[0] for person in group]
        create_image(names, coords, usernames, department, category, counter, background_path)
        counter += 1
    else:
        print(f"No coordinate layout or background found for group size {group_size}")



# Compute Frame Coordinates Function

TODO:
- This only caters to the one category which is bday.
- Create the other category as well